# Pixel classification on OpenCL-compatible GPUs
APOC is based on [pyclesperanto](https://github.com/clEsperanto/pyclesperanto_prototype) and [sklearn](https://scikit-learn.org/stable/).

Often, we want to include information from multiple channels (i.e., from different stainings) to be included in the prediction. APOC allows you to do just that. Let's start with loading an example image and some ground truth:

In [ ]:
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
import numpy as np
import apoc

image = imread('blobs.tif')
imshow(image)

You can now create some annotations.

*Note*: Even though your data may be multichannel data, annotations do not have to be multichannel. E.g., for a `[2, 256, 256]`-shaped image, creating an (scarcely) annotated image of shape `[256, 256]` works for APOC.

In [ ]:
if False: # you can use this to make manual annotations
    import napari

    # start napari
    viewer = napari.Viewer()
    napari.run()

    # add image
    viewer.add_image(image)

    # add an empty labels layer and keep it in a variable
    labels = np.zeros(image.shape).astype(int)
    viewer.add_labels(labels)
else:
    labels = imread('annotations.tif')

In [ ]:
manual_annotations = labels

from skimage.io import imshow
imshow(manual_annotations, vmin=0, vmax=3)

We now convert the input data into an artificial 2-channel data:

In [ ]:
channel_1 = image
channel_2 = (1 - image)**2

fig, axes = plt.subplots(ncols=2)
axes[0].imshow(channel_1)
axes[0].set_title('First channel')
axes[1].imshow(channel_2)
axes[1].set_title('Second channel')

multi_channel_image = np.stack([channel_1, channel_2])
print('Image data dimensions: ', multi_channel_image.shape)

## Training
We now train a PixelClassifier, which is under the hood a [scikit-learn RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). After training, the classifier will be converted to [clij-compatible OpenCL code](https://github.com/clEsperanto/clij-opencl-kernels) and save to disk under a given filename.

In [ ]:
# define features: original image, a blurred version and an edge image
features = "original gaussian_blur=2 sobel_of_gaussian_blur=2"

# this is where the model will be saved
cl_filename = 'my_model.cl'

apoc.erase_classifier(cl_filename)
clf = apoc.PixelClassifier(opencl_filename=cl_filename)
clf.train(features, manual_annotations, multi_channel_image)

## Prediction
The classifier can then be used to classify all pixels in the given image. Starting point is again, the feature stack. Thus, the user must make sure that the same features are used for training and for prediction.

In [ ]:
result = clf.predict(image=multi_channel_image)
imshow(result)

If we pass the data to the `clf.predict()` function in a wrong format - for instance, by passing single-channel data to a classifier that was trained on multi-channel data - we receive an error:

In [ ]:
results = clf.predict(image = channel_1)

To avoid this issue, we can use the `clf.info()` function to see which dimensions the classifer expects for the prediction. 

*Note*: For the classifier to work on new data, **only the first dimension** needs to match. In other words:

* A classifer trained on a `[2, 256, 256]` image will work for new data with dimensions `[2, 512, 512]`
* A classifer trained on a `[2, 256, 256]` image will **not** work for new data with dimensions `[1, 256, 256]`

In [ ]:
clf.info()